# YOLOv5

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

#### Clone the YOLO repository open-sourced by Ultralytics

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (CPU)


#### **Read and Complete the following tasks before running the next code block**

* upload **detect1.py** given with this notebook in the **yolov5** folder
* upload **best.pt** given with this notebook in the **yolov5** folder, also uploaded on the link https://drive.google.com/file/d/1x_PtbXkmbZFq6jwwx7RZGgm5JGlOKZvT/view?usp=sharing
* upload the **custom.yaml** file into the **yolov5** folder. The custom.yaml file is provided with the notebook

* The training dataset is uploaded at this link
https://drive.google.com/drive/folders/181lv3C9BJaI4M9YCLLTKzKaLVsyw9O3N?usp=sharing with the name **train_data**

* Copy the path of folder named "**train_data**"

* Open custom.yaml file and **update the variables** 'path', 'train' and 'val' in custom.yaml file according to your system paths
 

 
It should look something like this:

```
path: /content/drive/MyDrive/train_data/  # dataset root dir\
train: /content/drive/MyDrive/train_data/images/train  # train images (relative to 'path') 128 images\
val: /content/drive/MyDrive/train_data/images/val  # val images (relative to 'path') 128 images\
test:  # test images (optional)


nc: 1  # number of classes
names: ['defect']  # class names

```






Make sure that the GPU hardware accelerator is enabled \
Run the next cell to start training

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data custom.yaml --weights yolov5s.pt --cache

After training the weights file would generally appear in **/content/yolov5/runs/train/exp/weights/best.pt** but i have already trained on yolov5x at uploaded the weights file 

* Put the path of **best.pt** file that we uploaded earlied against the **--weights** agrument

* Put the path of **testing images** against the **--source** argument

In [ ]:
!python detect1.py --weights "/content/drive/MyDrive/prithvi_ai/yolo/best.pt" --img 640 --conf 0.3 --source "/content/drive/MyDrive/CV Hackathon/Testing Images" --save-txt --exist-ok

* Now the labels of the testing images will the stored here **/yolov5/runs/detect/exp/labels**

* Spot this **label folder**  by going on that path and give it variable name "**path**"

We will be using it in the following steps

In [6]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [7]:
label_path = '/content/drive/MyDrive/yolo/labels/' # path to labels that we just extracted needs to go here

arr = os.listdir(path)

Making csv file for bounding box coordinated

In [8]:
# this reads every text file and extracts x, y, w, h values and store it in an array
# later that array is converted to DataFrame

df = [ ]
for i in arr:
    name = i.split('.')[0]
    file=open(f"{label_path}{i}","r")
    temp = file.readlines()
    for j in temp:
        cls, x, y, w, h = j.split(' ')
        x, y, w, h = float(x), float(y), float(w), float(h)
        df.append([i, x, y, w, h])
    file.close()

In [15]:
# Making DataFrame from the array
box = pd.DataFrame(data = df, columns=['image_id', 'X', 'Y', 'W', 'H'])

# saving dataframe
box.to_csv('bounding_box.csv', index=False)

Making csv file for defect flag

In [10]:
img_path = '/content/drive/MyDrive/CV Hackathon/Testing Images/'   #path to the testing images needs to go here

temp = os.listdir(img_path)
temp1 = os.listdir(label_path)
df1 = []
for i in temp:
    name = i.split('.')[0]
    file = f'{name}.txt'
    if file in temp1:
        val = 1
    else:
        val = 0
    df1.append([i, val])

In [12]:
# Converting array to DataFrame
detect = pd.DataFrame(data=df1, columns=["images id", "defect_flag"])

# Saving DataFrame as csv file
detect.to_csv('defect_flag.csv', index=False)